<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.977 · Análisis de sentimientos y textos</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencias de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 1: Procesamiento y análisis de información textual

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos en el módulo 1. Concretamente trataremos 4 temas.

1. **Obtención de datos a partir de información textual**: Búsqueda de términos unipalabra, colocaciones y vectorizaciones de términos.
2. **Obtención de datos a partir de recursos externos al texto**: Similitud de significado con Wordnet y ConceptNet.
3. **Interpretación de los datos**: Detectar temas.  
4. **Predicción**: Clasificación de textos.
   

## 1. Obtención de datos a partir de información textual (5 puntos)

Primero, cargamos las librerías necesarias.


In [3]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
import pandas as pd
import gensim
from sklearn.cluster import KMeans
import numpy as np

Seguiremos utilizando el dataset ArticlesMarch2018.csv visto en el módulo 1. Si lo abrís veréis que tiene 15 campos (columnas). Esta vez, además del campo headline, que ya se utilizó en el notebook del módulo 1, también utilizaremos el campo snippet y keywords.

Recordemos el contenido de cada campo:

<ul><b>Headline</b>: titulares</ul>
<ul><b>Snippet</b>: contenido de las noticias encabezadas por los titulares</ul>
<ul><b>Keywords</b>: palabras más relevantes de las noticias según sus redactores</ul>


## Preparación de los datos

In [4]:
#Transformamos el dataset en un dataframe
df= pd.read_csv("ArticlesMarch2018.csv")

#Concatenemos los valores de los campos headline y snippet para obtener un texto plano.
headline_texts = " ".join(df.headline.tolist()) #text plano
snippet_texts = " ".join(df.snippet.tolist()) #text plano

#Preparamos las keywords

keywords = []

todelete = ['[', ']', '\'']

for l in df.keywords.tolist():
    for td in todelete:
        l = l.lower().replace(td, '')
    keywords = keywords + l.split(', ')

print("========= KEYWORDS ========")
print(keywords[:10])

#Creamos un corpus a partir del cual haremos entrenamiento de modelos. Es la unión de los textos del campo 
#headlines y los textos del campo snippets

sentences_for_modelling = df.headline.tolist() + df.headline.tolist() # lista



========= KEYWORDS ========
['bitcoin (currency)', 'electric light and power', 'united states economy', 'labor and jobs', 'productivity', 'united states defense and military forces', 'north korea', 'united states international relations', 'united states politics and government', 'embargoes and sanctions']


## 1.1 Encontrar términos unipalabra (0.5 puntos)

<b>Ejercicio 1</b>

<ul>
<li>Listar 100 términos unipalabra del campo <i>headline</i> que no sean stopwords.</li>
<li>Lo mismo pero con el campo <i>snippet</i>. Los términos tienen que aparecer en minúscula.</li>
</ul>
    
<b>En este ejercicio, consideramos términos unipalabra a los tokens que empiezan y acaban con un caracter alfabético (e.g: 'casa' (SÍ), @casa (NO), casa26 (NO))</b>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    <strong>Implementación:</strong><ul>Repasad en el módulo PLA1 la diferencia entre token y término.</ul>
    <ul>Utilizad el tokenizador del paquete NLTK y filtrad los stopwords también con NLTK</ul>
    <ul>Para saber si los términos unipalabra más frecuentes son suficientemente informativos, calculad su intersección con el campo <b>keywords</b></ul>
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

Contestad las siguientes preguntas, razonando las respuestas:

<ul><b>¿Cuál es el porcentaje de términos unipalabra que aparecen en el campo <i>snippet</i> y aparecen también en <i>headline?</i></b></ul>
<ul><b>¿Crees que los primeros 100 términos unipalabra más frecuentes hallados en el campo <i>snippet</i> son suficientemente informativos para hacer un análisis detallado?</b></ul>
<ul><b>¿Alguno de estos términos más frecuentes los añadirías a la lista de stopwords?</b></ul>
<ul><b>Hay entradas sin titular marcadas con la palabra 'unknown', que no queremos considerar. ¿Pondrías esta palabra en la lista de stopwords?<ul><b></b></ul>

In [ ]:
#############################################
# SOLUCIÓN PREGUNTA 1                       #
#############################################


In [ ]:
#############################################
# SOLUCIÓN PREGUNTA 2                       #
#############################################

## Respuesta a la pregunta 3

Escribe aquí


## Respuesta a la pregunta 4

Escribe aquí



## 1.2 Encontrar colocaciones (2.5 puntos)

<b>Ejercicio 2</b>

Calcularemos las colocaciones más relevantes de los <b>snippets</b> según las métricas: <b>PMI</b> y <b>Likehood Ratio</b>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong>  A partir del texto de los snippets, computaremos los bigramas y trigramas más probables de ser una colocación.

Para este apartado, habrá que cargar las librerías siguientes:

In [12]:
from nltk import pos_tag, word_tokenize
from nltk.collocations import *
import re

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Primer paso:
    
A partir del comando help(nltk.collocations.BigramAssocMeasures) explora la clase BigramAssocMeasures del módulo nltk.metrics.association y revisa las definiciones de las medidas de Likelihood Ratio (likelihood_ratio) y de Pointwise Mutual Information (pmi) en las secciones que se indican en el capítulo 5 del libro Foundations of Statistical Natural Language Processing (Manning & Schutze).
</div>

In [5]:
help(nltk.collocations.BigramAssocMeasures)

Help on class BigramAssocMeasures in module nltk.metrics.association:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |      n_ii counts (w1, w2), i.e. the bigram being scored
 |      n_ix counts (w1, *)
 |      n_xi counts (*, w2)
 |      n_xx counts (*, *), i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |          

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Segundo paso:
    Calcular los términos unipalabra del campo <b>snippets</b> (Ya hecho en 1.1)
</div>


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Tercer paso: Computa los mil mejores bigramas y trigramas a partir de los términos unipalabra. Tal com se ha visto en el módulo 1, utiliza la medida <b>PMI</b>.

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Cuarto paso: Computa los mil mejores bigramas y trigramas a partir de los términos unipalabra, pero con la <b>Likehood Ratio</b>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<b>Ejercicio 3</b>

Calcularemos las colocaciones más relevantes de los <b>snippets</b> pero con el módulo <b>Phrases</b> de Gensim

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Previamente cargaremos el módulo Phrases.

In [16]:
from gensim.models.phrases import Phrases

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Preprocesa el texto que utilizaremos para entrenar el modelo de detección de phrases. Al resultado le llamaremos <b><i>stream</i></b>. Recuerda que, como vimos en el apartado 3.3 del notebook del módulo, el stream tiene que ser una lista de listas. Cada lista contiene los términos de una frase: e.g. [['virtual', 'coins', 'real', 'resources'],  ['u.s', 'advances', 'military', 'plans', 'for', 'north', 'korea'],  ['mr', 'trump', 'and', 'the', 'very', 'bad', 'judge']]

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Recuerda que no queremos considerar la palabra <b>unknown</b>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Entrena el modelo para detectar phrases.

In [19]:
#############################################
# SOLUCIÓN                                  #
#############################################



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Utiliza el modelo para detectar las phrases de los snippets. Presenta las phrases que contengan más de una palabra (el caracter delimitador de Phrases por defecto es '_')

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################


<b>Ejercicio 4</b>

Evaluación de los métodos de extracción de colocaciones

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong>  Comparar los resultados de cada método (PMI, Log Likehood y Phrase) con las <b>Keywords</b>.
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    
Primer paso: Encontrar la intersección entre la suma de bigramas y trigramas calculados con el método PMI y la lista de keywords
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    
Segundo paso: Encontrar la intersección entre la suma de bigramas y trigramas calculados con el método Log Likehood y la lista de keywords</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    
Tercer paso: Encontrar la intersección entre la suma de bigramas y trigramas calculados con el método Phrase y la lista de keywords
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Análisis:</strong> Contesta las siguientes preguntas, razonando las respuestas:
<ul>¿Los resultados de Phrase se pueden evaluar comparándolos con los resultados del método PMI y Log Likehood?</ul>
    <ul>¿Qué mètodo crees que es mejor para sacar las colocaciones de los snippets?</ul>
</div>

## Respuesta pregunta 1

Escribe la respuesta



## Respuesta pregunta 2

Escribe la respuesta

<b>Ejercicio 5</b>

Mejora los métodos de extracción de colocaciones con <b>información lingüística</b>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong> Mejora la gramática propuesta en el mòdulo 1 para detectar n-gramas añadiendo otros patrones sintácticos, por ejemplo patrones que combinen adjetivos y nombres.
</div>

Recordemos la clasificación de etiquetas PoS.

<b>Etiquetas PoS</b>

<ul>
<li>DT: Determinante</li>
<li>JJ: Adjetivo</li>
<li>NN: Nombre en singular</li>
<li>NNS: Nombre en plural</li>
<li>VBD: Verbo en pasado</li>
<li>VBG: Verbo en gerundio</li>
<li>MD: Verbo modal</li>
<li>IN: Preposición</li>
<li>PRP: Pronombre</li>
<li>RB: Adverbio</li>
<li>CC: Conjunción coordinada</li>
<li>CD: Numeral</li>
</ul>

In [26]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Propuesta del alumno

### 1.3 Vectorizar términos (2 puntos)

<b>Ejercicio 6</b>: Entrenar un modelo word2vec de los titulares con los snippets (text_for_modelling)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
        Primer paso: preprocesa el texto que utilizaremos para entrenar el modelo, haciendo un <b>stream</b>, como has hecho en el ejercicio 3

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
Segundo paso: entrena un modelo Word2Vec a partir del texto preprocesado. Recordemos que el paquete <b>gensim</b> implementa un método para entrenar modelos Word2Vec.

In [27]:
#############################################
# SOLUCIÓN                                  #
#############################################



<b>Ejercicio 7</b>:
<li>
        <ul>a) Mostrar visualmente los vectores de los términos según el modelo Word2Vec mediante TSNE</ul>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
Cargamos los paquetes necesarios. Recuerda que tienes que obtener el vocabulario

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<li>
<ul>b) Obtén los 10 términos similares al término 'president', que no sean stopwords, según el modelo Word2Vec. Utiliza el método <b>most_similar</b> de la clase Word2Vec, que veréis en <a>https://radimrehurek.com/gensim/models/word2vec.html</a> y en <a>https://stackoverflow.com/questions/50275623/difference-between-most-similar-and-similar-by-vector-in-gensim-word2vec</a>. El parámetro de las n palabras más similares es topn (e.g: topn = 50)</ul>
</li>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<b>Ejercicio 8</b>:
<li>
 <ul>a) Cargamos un modelo w2vec pre-entrenado: </ul>
</li>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong> 
Cargamos un modelo Word2Vec entrenado previamente con todo el Google News dataset, con aproximadamente cien mil millones de palabras. <b>Atención: ¡el modelo ocupa casi 2 GB! y la carga puede tardar unos minutos.</b>
</div>

In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

b) Obtén los términos similares al término 'president' según el modelo pre-entrenado, que no sean stopwords. Como en el ejercicio 7, utiliza el método <b>most_similar</b>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

Si tuvieras que hacer un análisis de los artículos del New York Times, ¿utilizarías el modelo de Google News o crearías un modelo Word2Vec de cero, con las noticias de este periódico? Razona la respuesta

## Respuesta a la pregunta

Escribe la respuesta aquí



<b>Ejercicio 9</b>:
<li>
    <ul>A partir del modelo de Google News, demuestra el sentido de la palabra 'apple' comprobando las palabras con las cuales comparte contexto</ul>
    <ul>¿Es posible vincular un apple con Steve Jobs?</ul>
</li>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

## 2. Obtención de datos a partir de recursos externos al texto (1 punto)

En este apartado obtendremos datos de similitud semántica con los recursos <b>Wordnet</b> y <b>ConceptNet</b>

### 2.1 WordNet (0.5 puntos)


Accederemos a Wordnet a través de la librería nltk.

In [31]:
from nltk.corpus import wordnet as wn

El método de wordnet wup_similarity utiliza la métrica de similitud definida por la Wu and Palmer score en que el score (valor) depende de la distancia entre términos dentro del grafo de jerarquías de la ontología, que es el grafo que relaciona términos con sus hiperónimos e hipónimos (ver apartado 3.3.3 del documento "Cómo interpretar y analizar automáticamente la información textual" del módulo 1).

Además de la distancia entre dos nodos a través del grafo, la distancia también tiene en cuenta el nivel de jerarquía (más abstracto o menos abstracto), asignando más similitud entre nodos de niveles más bajos (menos abstracto) que entre nodos de niveles más altos (más abstracto).

<b>Ejercicio 10</b>: Comprueba que efectivamente la similitud de Wu and Palmer tiene el comportamiento esperado comparando las distancias entre pares de sinónimos en diferentes niveles de jerarquía (e.g. person vs human y kid vs child) y entre un término con un hipónimo y un hiperónimo (e.g: chair y table vs furniture).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong>  Recuerda que, como hemos visto en el módulo 1 en el apartado 4.1, para buscar dentro de wordnet puedes utilizar el método synset (e.g. wn.synset ('dog.n.01')) pasándole un string que contenga el nombre de la palabra que se busca, la categoría gramatical (sólo buscaremos nombres, i.e 'n') y el índice por si hay más de una entrada. Por lo tanto, en nuestro caso, siempre haremos la búsqueda wn.synset ('palabra.n.01') o wn.synsets ('palabra','n') para acceder a todas las entradas.
</div>

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

### 2.2 ConceptNet (0.5 puntos)

<strong>Ejercicio 11:</strong> A partir de la relación IsA de ConceptNet (http://api.conceptnet.io/r/IsA) extrae hiperónimos de los términos que has utilizado en el ejercicio anterior y compara sus distancias con los extraidos por WordNet. Explora también casos de phrases como 'United States' que con WordNet no se pueden explorar.
</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Para acceder a la api de ConceptNet cargamos la siguiente librería.

In [33]:
import requests

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Anàlisis:</strong> ¿Cuáles son las principales diferencias entre WordNet y ConceptNet?
</div>

## Respuesta a la pregunta

Escribe la respuesta aquí



## 3. Interpretación de los datos: detección de temas. (2 puntos)

En este apartado exploraremos cómo detectar temas en los documentos. Primero veremos cómo el clustering de las keywords de las noticias perfilan los temas. Utilizaremos el método de K-means, vectorizando las keywords con un vectorizador tf.idf.

En primer lugar, recogemos, para cada noticia, una lista de sus keywords.

In [36]:
df = pd.read_csv("ArticlesMarch2018.csv")
keywords_lists = df.keywords.tolist() #lista de keywords por noticia

keywords_lists = list(map(lambda x: list(map(lambda y: y.strip(' \'[]\''), x.split(','))), keywords_lists))

keywords_lists[:3]

[['Bitcoin (Currency)',
  'Electric Light and Power',
  'United States Economy',
  'Labor and Jobs',
  'Productivity'],
 ['United States Defense and Military Forces',
  'North Korea',
  'United States International Relations',
  'United States Politics and Government',
  'Embargoes and Sanctions',
  'Civilian Casualties',
  'Defense Department',
  'Joint Chiefs of Staff'],
 ['Trump',
  'Donald J',
  'Curiel',
  'Gonzalo P',
  'United States Politics and Government',
  'Decisions and Verdicts',
  'Courts and the Judiciary',
  'Mexico',
  'Border Barriers']]

Convertimos cada lista de keywords en una cadena de palabras; es decir, en un documento

In [37]:
keywords_docs = [" ".join(l) for l in keywords_lists ]

print(keywords_docs[:3])

['Bitcoin (Currency) Electric Light and Power United States Economy Labor and Jobs Productivity', 'United States Defense and Military Forces North Korea United States International Relations United States Politics and Government Embargoes and Sanctions Civilian Casualties Defense Department Joint Chiefs of Staff', 'Trump Donald J Curiel Gonzalo P United States Politics and Government Decisions and Verdicts Courts and the Judiciary Mexico Border Barriers']


### 3.1 K-means (1 punto)

<strong>Ejercicio 12:</strong> Clusteriza las keywords en 4 clusters

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
    
<strong>Implementación:</strong>Vectorizamos los documentos con un tf.idf vectorizer, que tiene 'word' como tokenizador

In [38]:
#############################################
# SOLUCIÓN                                  #
#############################################



<strong>Implementación:</strong> Visualiza las keywords en 4 clusters. Ten en cuenta que, en el paso anterior, los vectores ya están en una matriz

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<strong>Ejercicio 13:</strong> ¿La gráfica es suficientemente clara como para ver los términos que definen los temas? ¿Crees que aumentando el número de clusters se vería mejor? ¿O sería más esclarecedor listar los términos de cada cluster? Razona las respuestas

## Respuestas a las preguntas

Escribe las respuestas aquí



### 3.2 LDA (1 punto)

Recordemos que en el notebook del módulo 1 hemos visto la aplicación del método LDA para extraer temas de documentos

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong> Utilizando el método LDA, extrae temas a partir de los conjuntos de keywords de las noticias. Experimenta con el parámetro num_topics hasta encontrar un conjunto de temas que encuentres informativo, y asigna nombres a los temas encontrados. La construcción del modelo LDA puede tardar más de 10 minutos para algunos casos.
</div>

Preparamos, por cada noticia, la lista de sus keywords (keywords lists). Ver introducción apartado 3.

Cargamos el módulo necesario para utilizar el método LDA y recordamos la definición de la función que se utiliza en el notebook de módulo 1.

In [48]:
import gensim.corpora as corpora

def lda(terms):
    dictionary = corpora.Dictionary(terms)

    texts = terms

    corpus = [dictionary.doc2bow(text) for text in texts]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                               num_topics=10, 
                                               random_state=1,
                                               id2word = dictionary, 
                                               passes=500)

    return ldamodel



<strong>Ejercicio 14:</strong> Partiendo de la función LDA anterior explora modelos de temas a partir de la lista de keywords de las noticias. Utiliza diferentes valores para el campo num_topics.

In [49]:
#############################################
# SOLUCIÓN                                  #
#############################################



## 4. Predicciones (2 puntos)

La empresa S&S hizo la clasificación de los titulares TOP y NOTOP con titulares del New York Times durante la época Trump. Aquí comprobaremos si el clasificador es útil para clasificar titulares publicados ya en la era Biden. 


<strong>Ejercicio 15:</strong>Crea un modelo de clasificación de titulares a partir de los titulares del fichero 'NYT-Comment-Headlines.csv'

Preparamos los titulares y las etiquetas TOP y NOTOP para emparejarlos para hacer el entrenamiento

In [4]:
import pandas as pd


df = pd.read_csv('NYT-Comment-Headlines.csv', sep='\t')

data_headlines = []
data_labels = []

headlines = df['Headline'].tolist()
tags = df['Tag'].tolist()

for i in range(len(headlines)): 
    data_headlines.append(headlines[i]) 
    data_labels.append(tags[i])

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementación:</strong> Vuelve a entrenar un modelo de predicción para TOP o NO TOP a partir de la vectorización TfIdf de los titulares. Utiliza los términos detectados por el modelo de detección de phrases.
</div>

Cargamos las librerías necesarias para el entrenamiento y la utilización del modelo. Además, cargamos el modelo de detección de phrases.


In [88]:
import numpy as np
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from gensim.models.phrases import Phrases

model_phrases = Phrases.load('model_phrases')

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Paso 1: Construye los vectores TfIdf de los titulares, utilizando como términos las phrases detectadas.

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Paso 2: Entrena el modelo clasificador utilizando Logistic Regression com en el notebook del módulo 1.

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<strong>Ejercicio 16:</strong> Toma diez titulares actuales de la sección de política del New York Times. Comprueba si el clasificador funciona para estos titulares. ¿Hay temas, como el Covid-19, que influyen en los resultados del clasificador?

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">

Paso 1: Utiliza el modelo entrenado para predecir la categoría TOP o NO TOP de los titulares del New York Times que has escogido. Ver las palabras más informativas para cada clase (TOP y NO TOP).

In [ ]:
#############################################
# SOLUCIÓN                                  #
#############################################

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;"> Paso 2: Según el número de comentarios que tiene cada noticia razona si los resultados son los esperables y, si no es así, ¿qué aspectos crees que puede haber influido?

## Respuesta a la pregunta

Escribe la respuesta aqui

